In [1]:
import requests
import json
import pandas as pd
import os

from google.cloud import bigquery
from google.oauth2 import service_account

#external modules
import api_call_helpers as ach
import bigquery_functions as pp
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [2]:
# bigquery credentials
path = os.getcwd()
secrets_filename = 'bigquery_auth.json'

CREDENTIALS = service_account.Credentials.from_service_account_file(
    f'{path}/{secrets_filename}')
PROJECT_ID = 'riotdashboard-283414'
DATASET_ID = 'RIOT_DWH'
TABLE_ID = 'MATCH_STATISTICS'

In [3]:
# set up riot params
USERNAME = 'tanas0rn'
API_KEY = 'RGAPI-3f64c44b-25b4-43d0-ad7b-cf5a52fca5f1'
ORIGIN = "https://developer.riotgames.com"
ACCEPT_CHARSET = "application/x-www-form-urlencoded; charset=UTF-8"
ACCEPT_LANGUAGE = "de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7,en-GB;q=0.6"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"

headers = {
    "User-Agent": USER_AGENT,
    "Accept-Language": ACCEPT_LANGUAGE,
    "Accept-Charset": ACCEPT_CHARSET,
    "Origin": ORIGIN,
    "X-Riot-Token": API_KEY
}

# Connect to bigquery 
(get existing matches so that we dont insert matches already in the table)

In [4]:
# connect to BigQuery 
# get list of all existing matches/players
client = bigquery.Client(credentials= CREDENTIALS,project=PROJECT_ID)

query = """
SELECT matchID, summonername
FROM RIOT_DWH.MATCH_STATISTICS;
"""

query_job = client.query(query)
results = query_job.result() 

# Query riot api

In [5]:
# get summoner info (account ID etc)
encrypted_account_id = ach.get_encrypted_accid(USERNAME,headers)

In [6]:
# get match list for summoner
match_list = ach.get_match_list_by_accid(encrypted_account_id,headers)

Total 255 Matches Retrieved


In [7]:
# get all of the match IDs
match_ids = [item['gameId'] for item in match_list]
match_id = match_ids[0]

In [30]:
# get match stats
matchstats = []
for match_id in match_ids[:50]:
    stats = ach.get_match_stats(USERNAME,match_id,headers)
    
    matchstats.append(stats)

4718627204
4718642110
4645165732
4645161532
4645044253
4644927812
4644845260
4644649846
4644522144
4644247184
4637451701
4624982041
4619374728
4613342684
4429840064
4428341149
4428195686
4426871811
4425498151
4418520385
4418495755
4415740482
4415652050
4412372386
4412248032
4406574875
4403476468
4401712288
4400454500
4400482048
4400450483
4399317507
4397285400
4397248242
4394734741
4392927640
4392920807
4376714718
4374308739
4372868644
4372750341
4372381033
4372288812
4372200276
4372128420
4371912223
4371848844
4371750360
4370265465
4370148195


# Data preprocessing (to tuples + remove unwanted fields)

df = pd.DataFrame(matchstats)
for col in df.columns:
    print(col)

In [20]:
keys_to_remove = (
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6',
    'playerScore0',
    'playerScore1',
    'playerScore2',
    'playerScore3',
    'playerScore4',
    'playerScore5',
    'playerScore6',
    'playerScore7',
    'playerScore8',
    'playerScore9',
    'perk0',
    'perk0Var1',
    'perk0Var2',
    'perk0Var3',
    'perk1',
    'perk1Var1',
    'perk1Var2',
    'perk1Var3',
    'perk2',
    'perk2Var1',
    'perk2Var2',
    'perk2Var3',
    'perk3',
    'perk3Var1',
    'perk3Var2',
    'perk3Var3',
    'perk4',
    'perk4Var1',
    'perk4Var2',
    'perk4Var3',
    'perk5',
    'perk5Var1',
    'perk5Var2',
    'perk5Var3',
    'perkPrimaryStyle',
    'perkSubStyle',
    'statPerk0',
    'statPerk1',
    'statPerk2',
    'firstInhibitorKill', 
    'firstInhibitorAssist'
)

In [21]:
for match in matchstats:
    pp.entries_to_remove(keys_to_remove,match)

In [22]:
#convert to tuples
matchstats_tuples = [tuple(d.values()) for d in matchstats]

In [52]:
matchstats_str = []
for match in matchstats_tuples[25:26]:
    matchstats_str.append(str(match))

In [53]:
matchstats_insert = ",".join(matchstats_str)

# Insert data into bigquery table

In [55]:
# Insert rows to table
pp.insert_rows_to_tables(credentials = CREDENTIALS,
                          project_id = PROJECT_ID,
                          dataset_id = DATASET_ID,
                          table_id = TABLE_ID,
                          insert_rows = matchstats_insert)